In [138]:
import pandas as pd 
import numpy as np
import networkx as nx
import scipy.sparse as sp
from sklearn import preprocessing
import matplotlib.pyplot as plt
import csv
import torch
import itertools 

### Useful functions: 

In [139]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

### Construction of the node-node-label graph : 

In [140]:
data_name = "BlogCatalog"
edges_file = data_name + "/edges.csv"
node_label_file = data_name + "/group-edges.csv"
label_occ_file = data_name + "/label_co-occurences.csv"
nnlg_file = data_name + "/edges_node_node_label.csv"
llng_file = data_name + "/edges_label_label_node.csv"
label_raw, nodes = [], []
with open(node_label_file) as file_to_read: 
    while True:
        lines = file_to_read.readline()
        if not lines:
            break 
        node, label = lines.split(",")
        label_raw.append(int(label))
        nodes.append(int(node))

label_raw = np.array(label_raw)
nodes = np.array(nodes)
unique_nodes = np.unique(nodes)
labels = np.zeros((unique_nodes.shape[0], 39))
for l in range(1, 40, 1):
    indices = np.argwhere(label_raw == l).reshape(-1)
    n_l = nodes[indices]
    for n in n_l:
        labels[n-1][l-1] = 1

In [141]:
label_nodes = label_raw + unique_nodes.shape[0] 
n_n_l_nodes = np.concatenate((unique_nodes, np.unique(label_nodes)))

In [142]:
df = pd.DataFrame(list())
df.to_csv(nnlg_file)

In [143]:
f = open(nnlg_file, "r+")
file_to_read = open(edges_file, "r")
f.writelines(file_to_read.readlines())

In [144]:
a = np.dstack((nodes, label_nodes)).reshape(label_nodes.shape[0],2)
e = ["\n"] + [",".join(item)+"\n" for item in a.astype(str)]

In [145]:
f.writelines(e)
f.close()

In [146]:
nnlg_file = "BlogCatalog/edges_node_node_label.csv"

In [147]:
nnl_graph = nx.read_edgelist(nnlg_file, delimiter = ",", nodetype = int)
E = nx.adjacency_matrix(nnl_graph, nodelist = n_n_l_nodes)

In [148]:
main_graph = open(edges_file, "rb")
G = nx.read_edgelist(main_graph, delimiter = ",", nodetype = int)
A = nx.adjacency_matrix(G, nodelist = unique_nodes)
A = sp.coo_matrix(A.todense())

In [149]:
# The feature matrix of the common nodes
X = sp.csr_matrix(A)

In [150]:
A_tilde = normalize(A + sp.eye(A.shape[0]))

### Construction of the label-label-node graph : 

In [151]:
edges = []
list_edges = []
for k in range(labels.shape[0]):
    indices = np.argwhere(labels[k] == 1).reshape(-1)
    if indices.shape[0]>1:
        for subset in itertools.combinations(indices, 2): 
            if (list(subset) not in list_edges) or ([subset[1], subset[0]] not in list_edges):
                list_edges.append([subset[0]+labels.shape[0], subset[1]+labels.shape[0]])# check if the common nodes should be before or after the label nodes
                edges.append(str(subset[0]+1 + labels.shape[0]) + "," + str(subset[1] +1 + labels.shape[0]) + "\n")

In [152]:
df = pd.DataFrame(list())
df.to_csv(label_occ_file)

In [153]:
label_file = open(label_occ_file, "r+")

In [154]:
label_file.writelines(edges)
label_file.close()

In [155]:
unique_label_ID = np.arange(1,40) + labels.shape[0]
label_file = open(label_occ_file, "rb")
label_graph = nx.read_edgelist(label_file, delimiter = ",", nodetype = int)
C = nx.adjacency_matrix(label_graph, nodelist = unique_label_ID)
label_file.close()

In [156]:
C_tilde = normalize(C + sp.eye(C.shape[0]))

In [157]:
# The feature matrix of the label nodes
Y = X[:39]

In [158]:
labels_ind = label_raw + labels.shape[0]
a_1 = np.dstack((labels_ind,nodes)).reshape(labels_ind.shape[0],2)
e_1 = ["\n"] + [",".join(item)+"\n" for item in a_1.astype(str)]

In [159]:
file = open(label_occ_file, "r+")
file.writelines(e_1)
file.close()

In [160]:
f_1 = open(label_occ_file, "rb")
l_l_n_nodes = np.concatenate((np.unique(nodes),np.unique(labels_ind)))
lln_graph = nx.read_edgelist(f_1, delimiter = ",", nodetype = int)
F = nx.adjacency_matrix(lln_graph, nodelist = l_l_n_nodes)
F = sp.coo_matrix(F.todense())
f_1.close()

In [161]:
X_star = sp.vstack((X,Y))
Y_star = sp.vstack((Y,X))

In [162]:
E = normalize(E + sp.eye(E.shape[0]))

In [163]:
E_tilde = E[:len(unique_nodes)]

In [164]:
F = normalize(F + sp.eye(F.shape[0]))

In [165]:
F_tilde = F[len(unique_nodes):]

In [166]:
X

<10312x10312 sparse matrix of type '<class 'numpy.intc'>'
	with 667966 stored elements in Compressed Sparse Row format>

In [167]:
Y 

<39x10312 sparse matrix of type '<class 'numpy.intc'>'
	with 3408 stored elements in Compressed Sparse Row format>

In [168]:
X_star 

<10351x10312 sparse matrix of type '<class 'numpy.intc'>'
	with 671374 stored elements in Compressed Sparse Row format>

In [169]:
Y_star

<10351x10312 sparse matrix of type '<class 'numpy.intc'>'
	with 671374 stored elements in Compressed Sparse Row format>

In [170]:
F_tilde

<39x10351 sparse matrix of type '<class 'numpy.float64'>'
	with 14515 stored elements in Compressed Sparse Row format>

In [171]:
E_tilde

<10312x10351 sparse matrix of type '<class 'numpy.float64'>'
	with 692754 stored elements in Compressed Sparse Row format>

In [172]:
C_tilde

<39x39 sparse matrix of type '<class 'numpy.float64'>'
	with 1269 stored elements in Compressed Sparse Row format>

In [173]:
# indices for the train/validation/test 
indices = np.arange(A.shape[0]).astype('int32')
idx_train = indices[:A.shape[0] // 3]
idx_val = indices[A.shape[0] // 3: (2 * A.shape[0]) // 3]
idx_test = indices[(2 * A.shape[0]) // 3:]

In [174]:
# Convert everything to tensors for the training 
idx_train = torch.LongTensor(idx_train)
idx_val = torch.LongTensor(idx_val)
idx_test = torch.LongTensor(idx_test)
X_star = torch.FloatTensor(np.array(X_star.todense()))
Y_star = torch.FloatTensor(np.array(Y_star.todense()))
C_tilde = torch.FloatTensor(np.array(C_tilde.todense()))
E_tilde = torch.FloatTensor(np.array(E_tilde.todense()))
F_tilde = torch.FloatTensor(np.array(F_tilde.todense()))

In [175]:
A_tilde = torch.FloatTensor(np.array(A_tilde.todense()))

In [176]:
labels = torch.FloatTensor(labels)

In [177]:
X_star.shape

torch.Size([10351, 10312])

### Let's get down to the training  

In [178]:
from Code.models import High_Layer, Low_Layer
import time
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

In [179]:
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=300,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.02,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=0,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=400,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')
parser.add_argument('-f')

args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [180]:
def threshold(output):
    output[output > 0.5] = 1
    output[output <= 0.5] = 0
    return output

In [181]:
def train_high_layer(Y_star):
    high_layer = High_Layer(nfeat = A_tilde.shape[0],
                            nhid = args.hidden,
                            nclass = C_tilde.shape[0],
                            dropout = args.dropout)
    optimizer_highLayer = optim.SGD(high_layer.parameters(), lr = args.lr, weight_decay = args.weight_decay)
    high_layer.train()
    optimizer_highLayer.zero_grad()
    Y_new = high_layer(Y_star, F_tilde, C_tilde)
    print(Y_new.shape)
    # Calculate the train loss (Cross-Entropy)
    truth = torch.LongTensor(np.arange(39))
    loss = F.cross_entropy(Y_new, truth) 
    print(type(loss))

    loss.backward()
    
    
    optimizer_highLayer.step()
    
    
    return Y_new, loss, high_layer

In [182]:
train_high_layer(Y_star)

torch.Size([39, 39])
<class 'torch.Tensor'>


(tensor([[-3.7474, -3.6812, -3.6715,  ..., -3.6148, -3.7054, -3.7793],
         [-3.7458, -3.6824, -3.6720,  ..., -3.6155, -3.7092, -3.7809],
         [-3.7455, -3.6824, -3.6695,  ..., -3.6147, -3.7074, -3.7811],
         ...,
         [-3.7408, -3.6833, -3.6796,  ..., -3.6199, -3.7095, -3.7812],
         [-3.7585, -3.6853, -3.6993,  ..., -3.6242, -3.6913, -3.7762],
         [-3.7043, -3.6595, -3.5652,  ..., -3.3994, -3.6774, -3.8317]],
        grad_fn=<LogSoftmaxBackward>),
 tensor(3.6708, grad_fn=<NllLossBackward>),
 High_Layer(
   (gc1): GraphConvolution (10312 -> 400)
   (gc2): GraphConvolution (400 -> 39)
 ))

In [183]:
def accuracy_sample_class(output, labels):
    """ 
    output is of shape (N,C)
    Labels is of shape (N,C)
    Result : acc gives the accuracy computed according to the sample-class view
    """
    N = labels.shape[0]
    C = labels.shape[1]
    corr = np.sum(np.equal(output, labels))
    # corr is the number of equal elements between labels and output and thus the number of correctly classified 
    # labels for each sample 
    acc = corr/(N*C)
    return acc

In [184]:
def train_low_layer(X_star):
    low＿layer = Low_Layer(nfeat = A_tilde.shape[0],
                           nhid = args.hidden,
                           nclass = C_tilde.shape[0],
                           dropout = args.dropout)
    optimizer_lowLayer = optim.SGD(low_layer.parameters(), lr = args.lr, weight_decay = args.weight_decay)
    low_layer.train()
    optimizer_lowLayer.zero_grad()
    X_new = low_layer(X_star, E_tilde, A_tilde)
    X_new = X_new.detach()
    # Calculate the train loss (Binary Cross Entropy)
    loss_train = torch.from_numpy(np.array((1/39)*np.sum([F.binary_cross_entropy_with_logits(X_new[idx_train][:,i], labels[idx_train][:,i]) for i in range(C_tilde.shape[0])])))
    loss_train.requires_grad = True
    acc_train = accuracy_sample_class(threshold(X_new.detach().numpy()[idx_train]), labels.detach().numpy()[idx_train])
    loss_train.backward()
    optimizer_lowLayer.step()
    
    loss_val = (1/39)*np.sum([F.binary_cross_entropy_with_logits(X_new[idx_val][:,i], labels[idx_val][:,i]) for i in range(C_tilde.shape[0])])
    
    return X_new, loss_train, loss_val, acc_train, low_layer

In [185]:
train_low_layer(X_star)

C:\Users\HSAIN KENZA\anaconda3\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(tensor([[0.4561, 0.4581, 0.5021,  ..., 0.4336, 0.4872, 0.4906],
         [0.4555, 0.4655, 0.5350,  ..., 0.4103, 0.4911, 0.5075],
         [0.4659, 0.4699, 0.5130,  ..., 0.4316, 0.4755, 0.5042],
         ...,
         [0.4512, 0.4549, 0.5004,  ..., 0.4123, 0.5075, 0.4979],
         [0.4376, 0.4386, 0.4802,  ..., 0.3899, 0.5290, 0.4579],
         [0.4656, 0.3896, 0.5279,  ..., 0.4221, 0.5168, 0.4122]]),
 tensor(0.9493, dtype=torch.float64, requires_grad=True),
 0.9502288622733874,
 0.5485627746320211,
 Low_Layer(
   (gc1): GraphConvolution (10312 -> 400)
   (gc2): GraphConvolution (400 -> 39)
 ))

In [194]:
# global train function for having a loop 
# epochs M N for
def global_train(epochs, M, N, Y_star, X_star):
    Y_star1 = Y_star
    X_star1 = X_star
    for i in range(epochs):
        Y_new, loss_hl, high_layer = train_high_layer(Y_star1)
        X_new, loss_train_ll, loss_val_ll, acc_train_ll, low_layer = train_low_layer(X_star1)
        
        
        X_new = X_new.detach()
        Y_new = Y_new.detach()
        
        if i%M:
            X_star1 = np.concatenate((Y_new, X_new),axis = 1)
            
        if i%N:
            Y_star1 = np.concatenate((X_new, Y_new), axis = 1)
            
        X_star1 = X_star1.detach()
        Y_star1 = Y_star1.detach()
        
        global_loss_train = loss_hl + loss_train_ll 
        # global loss function = combine the two loss functions
        # optimizer for global loss function
        # params = list(high_layer.parameters()) + list(low_layer.parameters())
        global_optimizer = optim.SGD(low_layer.parameters(), lr = args.lr, weight_decay = args.weight_decay)
        global_optimizer.zero_grad()
        
        if i == epochs-1:
            global_loss_train.backward()
        else:
            global_loss_train.backward(retain_graph = True)
   
        print(i)
        global_optimizer.step()

In [195]:
global_train(5, 2,3, Y_star, X_star)

torch.Size([39, 39])
<class 'torch.Tensor'>


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.